## WIP 3D SIM reconstruction template

This is a work in progress template for running 3D SIM reconstruction based on the Janelia Python and c SIM code written by David Hoffman and Lin Shao

## 1.  Define code paths
Currently we hard code these and they need to be modified to run on different machines.  In the future we may move to a more intelligent approach like always having code exist beside the notebooks and using relative imports.  

In [6]:
%pylab inline

import tifffile as tif
import os
import glob
%load_ext autoreload
%autoreload 
import shutil

# NOTE: BN the below code is from the legacy notebooks.  I am leaving it here for now, as it may be needed on some machines. 
if 'C:\\Users\\Cryo SIM-PALM\\Documents\\GitHub' in sys.path:
    sys.path.remove('C:\\Users\\Cryo SIM-PALM\\Documents\\GitHub')
else:
    pass

computer = 'bnort'

import sys

if computer == 'default':
    sys.path.insert(1, 'Y:\Cryo_data2\Data Processing Notebooks')
    sys.path.insert(1, 'Y:\Cryo_data2\Data Processing Notebooks\Scripts')
elif computer == 'bnort':
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\janeliaversion_tests\scripts\Scripts')
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\janeliaversion_tests\scripts')
else:
    pass

import dphutils 
from simrecon_utils import simrecon, split_process_recombine

# import dask
import dask
from dask.diagnostics import ProgressBar

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# SETUP HOME DIRECTORY and DATA PROCESSING NOTEBOOKS and OTF directory:

BN:  I think we should set up all three directories here so user only has to modify in one place.

Right now we leave in paths for the current test machines but in the future may move to a more intelligent approach (for example user chooses paths with dialog box, paths stored in configuration)

Note:  I don't see the 'data processing notebooks' location set or used anywhere.  Is this a legacy thing that is no longer needed? 

Home should start from {Hesslab(\\prfs.hhmi.org)} e.g. home = r'Y:\Cryo_data2\ORCA_data\3D SIM Cells'

Data Processing notebooks should be here: 'Y:\Cryo_data2\Data Processing Notebooks'

## Additonal legacy notes

There was an old note "OTF folder should be placed inside Data processing notebooks"

(BN I don't think this has to be the case, because there are many notebook that define the full OTF path)

OTF Folder directory should be e.g.:   Y:\Cryo_data2\Data Processing Notebooks\SIM PSFs OTFs

Raw data from V-SIM data acquisition should be placed inside a dated folder here:   'Y:\Cryo_data2\ORCA_data\3D SIM Cells' 

with data file directory structure e.g.: 'Y:\Cryo_data2\ORCA_data\3D SIM Cells\20240322\488 nm 5 phases 0.81 NA Linear SIM_cam1_0.mrc'

In [10]:
if computer == 'default': 
    home = r'Y:\Cryo_data2\488nm comparison TILED VS Non-Tiled at different base kwarg settings'
    otf_path = r'Y:\Seyforth\Data For Brian\Cryo-SIM Scope #2 Data (James System)\PSFs (best PSFs and examples of bad ones)\BEAD 2 - NON-AR 1.2W 25ms retake_20240503_170242 BEST PSF!!\computed_OTF_folder'
elif computer == 'bnort':
    home = r'D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian'
    otf_path = r'D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\OTF\BEAD 2 - NON-AR 1.2W 25ms retake_20240503_170242 BEST PSF!!\computed_OTF_folder'

OTFpath = os.path.join(otf_path,"*{}*.mrc")
OTFs = {wl : [path for path in glob.iglob(OTFpath.format(wl))] for wl in (560, 532, 488, 642)}
OTFs

{560: [],
 532: [],
 488: ['D:\\Janelia\\Data 2024-06-06\\Wiener, gammaApo and SupressR parameter testing\\OTF\\BEAD 2 - NON-AR 1.2W 25ms retake_20240503_170242 BEST PSF!!\\computed_OTF_folder\\488nmLinOTF0.mrc'],
 642: []}

## Set default params

Here we set the default params that will be used if none of them are overwritten.

Note that before calling the non-tiled and/or tiled processing code a small subset of parameters will be overwritten to values optimized for non-tiled/tiled processing

In [7]:
# David Solecki's suggestion: wiener=0.007 gammaApo=0.7 and suppressR=15
# original values by D.Hoffman: wiener=0.001 gammaApo=0.1 and suppressR=1.5

base_kwargs = dict(
                    nphases=5,
                    ndirs=3,
                    angle0= 1.29,
                    negDangle=True,              # James made False to try experiment
                    na= 0.85,
                    nimm= 1.0,
                    zoomfact= 2.0, 
                    background= 100.0,           # james experiment was 100.0
                    wiener= 0.007,
                    fastSIM=True,
                    otfRA= True,
                    dampenOrder0=True,
                    k0searchall=True,
                    equalizez=True,
                    preciseapo=True,
                    gammaApo=0.7,
                    suppressR=15.0
                )

def return_wl_otfs(path):
    if "488 Exc 532 Em" in path:
        wl = 532
    elif "488 Exc 642 Em" in path:
        wl = 642
    elif "532 Exc 561 Em" in path:
        wl = 561
    elif "560 nm" in path:
        wl = 560
    elif "488 nm" in path:
        wl = 488
    elif "532 nm" in path:
        wl = 532
    elif "642 nm" in path:
        wl = 642

    else:
        raise RuntimeError("no matching filename wavelength found, fix directory or filename or code")
    return wl, OTFs[wl]

## remove old processed data

This is commented out, but I assume we comment back in if we want to erase the previous run

In [5]:
'''python
# clear processed
for path in glob.glob(home + "/*/*proc *.mrc"):
    os.remove(path)
for path in glob.glob(home + "/*/*proc *.txt"):
    os.remove(path)
'''

'python\n# clear processed\nfor path in glob.glob(home + "/*/*proc *.mrc"):\n    os.remove(path)\nfor path in glob.glob(home + "/*/*proc *.txt"):\n    os.remove(path)\n'

# Define process function defined to use DASK and simrecon package

This function is annotated as a dask ```delayed``` function.  Which means it will not be called right away but put in a queue to call using dask. 

In [8]:
@dask.delayed
def process(sim_kwargs, output_file_name):
    sim_output = simrecon(**sim_kwargs)
    with open(output_file_name.replace(".mrc", ".txt"), "w") as myfile:
        myfile.write(str(sim_kwargs))
        myfile.write("\n".join(sim_output))
    return output_file_name

# Cell below checks for raw data which has already been processed using full frame SIM reconstruction algorithm

In [11]:
done_already = set(glob.iglob(home + "/**/*proc*.mrc", recursive = True))
done_already;
done_already = set()     # do this if want to re-process
print('Data processed already: \n')

for i in done_already:
    print(i + '\n')

Data processed already: 



# Full frame Simrecon SIM image reconstruction, recon parameters: 

Set up full frame processing.  Note that we override gammaApo, supressR and wiener parameters.  Also note the code will not be run right away (just put in a dask queue)

In [12]:
# RUN ME!
from contextlib import suppress


to_process = []

gammaApo = 0.3
suppressR = 10
wiener = 0.001

base_kwargs.update(dict(gammaApo=gammaApo, suppressR=suppressR, wiener=wiener))   # default Full frame Recon. parameters

user_text = 'gApo_'+str(gammaApo)+'_supR_'+str(suppressR)+'_w_'+str(wiener) 

for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    wl, otfs = return_wl_otfs(raw)
    for otf in otfs:
        if "proc" not in raw:
            print('Data to process ' +str(raw))
            # Test with actually measured OTF
            sim_kwargs = dict(                                                                                                            
                input_file= raw,
                otf_file= otf,
                ls= (wl/1000)/2/0.81,)
            
            sim_kwargs.update(base_kwargs)
            OTF_filename = os.path.split(otf)[1].split('.')[0]
            print('\nOTF filename: ' +str(OTF_filename))
                
            #create processed file output name
            sim_kwargs["output_file"] = sim_kwargs["input_file"].replace(".mrc", '_proc' + OTF_filename + '_' 
                                                                         + user_text + ".mrc")
                
            if sim_kwargs["output_file"] not in done_already:
                print("\n\nSIM recon Data to save:" + '\n' + str(sim_kwargs["output_file"]) + '\n')
                to_process.append(process(sim_kwargs, sim_kwargs["output_file"]))

Data to process D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc

OTF filename: 488nmLinOTF0


SIM recon Data to save:
D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc488nmLinOTF0_gApo_0.3_supR_10_w_0.001.mrc



# Running cell below shows all files being processed for full frame reconstruction and progress using DASK

This cell will start and run the processing.  We use dask to compute our processing queue that was set up in the cell above.

In [13]:
with ProgressBar():
    print(to_process)
    out_names = done_already.update(*dask.compute(to_process))

[Delayed('process-c89ec95a-6a43-4767-9347-b07036a7d7b9')]
[########################################] | 100% Completed |  1min  1.6s


# cell below checks for data already processed using the tiledreconstruction method, cell below that creates list of data to be set to done already, i.e. already processed


In [14]:
for path in glob.glob(home + "/*/*_tile64_pad32*"):
    done_already.add(path.replace("_tile64_pad32", "") + "__split__")

In [15]:
[path for path in done_already if "_tile64_pad32" in path]

[]

In [16]:
for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    
    print(raw)

D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc
D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc488nmLinOTF0_gApo_0.3_supR_10_w_0.001.mrc
D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc__488nmLinOTF0_gApo_0.3_supR_10_w_0.001.mrc


In [17]:
print(home)
print(glob.glob(home + "/**/*SIM_*.mrc"))

D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian
['D:\\Janelia\\Data 2024-06-06\\Wiener, gammaApo and SupressR parameter testing\\488nm comparison Brian\\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc', 'D:\\Janelia\\Data 2024-06-06\\Wiener, gammaApo and SupressR parameter testing\\488nm comparison Brian\\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc488nmLinOTF0_gApo_0.3_supR_10_w_0.001.mrc', 'D:\\Janelia\\Data 2024-06-06\\Wiener, gammaApo and SupressR parameter testing\\488nm comparison Brian\\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc__488nmLinOTF0_gApo_0.3_supR_10_w_0.001.mrc']


# Cell below runs tiled reconstruction of SIM images

In [18]:
%%time

gammaApo = 0.2
suppressR = 1.5
wiener = 0.001

base_kwargs.update(dict(gammaApo=gammaApo, suppressR=suppressR, wiener=wiener)) # default tiling Recon. parameters

user_text = 'gApo_'+str(gammaApo)+'_supR_'+str(suppressR)+'_w_'+str(wiener)

for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    wl, otfs = return_wl_otfs(raw)
    print(raw)
    
    for otf in otfs:
        if "proc" not in raw:                  #checks for if data was already processed
            if "_tile64_pad32" not in raw:       #checks for if data was already processed using tiling method
                print("\nFile to be processed: " + str(raw))
                # Test with actually measured OTF
                sim_kwargs = dict(
                    input_file= raw,
                    otf_file= otf,
                    ls= (wl/1000)/2/0.81,
                )
                sim_kwargs.update(base_kwargs)
                
                OTF_filename = os.path.split(otf)[1].split('.')[0]
                
                #create processed file output name
                out_name = sim_kwargs["output_file"] = sim_kwargs["input_file"].replace(".mrc",'_proc_' + OTF_filename + '_' +
                                                                          user_text + ".mrc")
                
                # perform reconstruction and output recon image file and text file
              
                out_name += "__split__"
                
                if out_name not in done_already:
                    #print(sim_kwargs["input_file"])
                    %time sim_output = split_process_recombine(sim_kwargs["input_file"], 64, 32, sim_kwargs)
                    # HAD TO EDIT simrecon_utils.py lines 1488 and 1490 and comment out mrc.close() as it was failing.
                    with open(sim_output[0].replace(".mrc", ".txt"), "w") as myfile:
                        myfile.write(str(sim_kwargs))
                        myfile.write("\n" + "-" * 80 + "\n")
                        myfile.write("\n".join(sim_output[1]))

                done_already.add(out_name)
                for path in glob.glob(home + "/*/*_tile64_pad32*"):
                    done_already.add(path.replace("_tile64_pad32", "") + "__split__")

D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc

File to be processed: D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc



['D:/tmptagdl762\\sub_image000000_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000001_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000002_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000003_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000004_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000005_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000006_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000007_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000008_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000009_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000010_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000011_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc', 'D:/tmptagdl762\\sub_image000012_97b4961ff3b60fbd3ebed336686c5fa4_proc.mrc

MRC Raw data file is still open
MRC temp data file is still open
MRC Raw data file is now closed
MRC temp data file is now closed
Wall time: 3min 54s
D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc488nmLinOTF0_gApo_0.3_supR_10_w_0.001.mrc
D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc__488nmLinOTF0_gApo_0.3_supR_10_w_0.001.mrc
Wall time: 3min 54s


###### from simrecon_utils import process_txt_output, plot_params

In [ ]:
dst = "Tiled Parameters/"
try:
    os.mkdir(dst)
except FileExistsError:
    pass
for raw in glob.iglob(home + "/*/*SIM*tile*.txt"):
    with open(raw) as f:
        fig, axs = plot_params(process_txt_output("".join(f.readlines())))
        fig.savefig(dst + os.path.abspath(raw).split(os.path.sep)[-2] + ".png", dpi=300, bbox_inches="tight")

In [ ]:
print("done!")

# Renaming 3D SIM files for DropBox

In [ ]:
for wl in ("488 nm", "532 nm", "560 nm", "642 nm"):
    curdir = os.path.split(os.path.abspath(os.curdir))[1]
    glob_str = "F:/ORCA_Data/{}/3D SIM Cells/*{}*/*{}.mrc".format(curdir, "", "{}*SIM*proc*".format(wl))
    for path in sorted(glob.glob(glob_str)):
        print(path)
        path_head, path_tail = os.path.split(path)
        path_head_new, path_tail_new = os.path.split(path_head)
        file_root = path_tail_new.split("_")[0]+' '+path_tail_new.split("_")[1]

        OTF = path_tail.split('_OTF')[-1]
        proc = OTF.split('_proc')[-1]
        if proc == '.mrc':
            OTF = proc = OTF.split('_proc')[0] + '.mrc'
        new_path = os.path.join(path_head_new, file_root + ' wl'+wl.split(' ')[0]+' proc_with_OTF' + OTF)
        print(new_path)
        print('')
        
        shutil.copyfile(path,new_path)